In [15]:
# Set logging for the queries
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)
logging.getLogger("langchain.retrievers.re_phraser").setLevel(logging.INFO)

In [1]:
import yaml
from dotenv import load_dotenv
import os

# Mở và đọc file YAML
with open('configs/base.yaml', 'r') as file:
    configs = yaml.safe_load(file)

load_dotenv() 
api_key = os.getenv('API_KEY')

# Load model

In [ ]:
# remote model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
        model=configs['model']['model_id'],
        temperature=configs['model_params']['temperature'],
        max_tokens=configs['model_params']['max_tokens'],
        timeout=configs['model_params']['timeout'],
        max_retries=configs['model_params']['max_retries'],
        api_key=api_key,
        base_url=configs['server']['base_url'],
    )

# Load embedding

In [6]:
from langchain_community.embeddings import GPT4AllEmbeddings
embedding = GPT4AllEmbeddings(model_name = configs['embedding']['model_name'])

# Create DB

In [34]:
# from utils.helper_func import pretty_print_docs

In [7]:
from modules.VectorDB import VectorDB

vector_db = VectorDB(embedding = embedding,
                     chunk_size=configs['vector_db']['chunk_size'],
                     chunk_overlap=configs['vector_db']['chunk_overlap'])

db = vector_db.create_db(configs['data_source'])
base_retriever = db.as_retriever(search_kwargs=configs['retrieval']['search_kwargs'])

In [ ]:
from modules.QueryRetriever import RetrieverSystem
retriever_system = RetrieverSystem(llm, embedding)

retriever = retriever_system.create_retriever(
    base_retriever,
    {
        "type": "ensemble",
        "retrievers": [
            # {"type": "rephrase"},
            {"type": "multiple_query"},
            {"type": "contextual_compression", 
             "compressor": {"type": "rerank"}}
        ],
        "weights": [0.3, 0.3, 0.4]
    }
)

# Inference

In [ ]:
# INFERENCE CHAT
from scripts.Chat import Chat
prompt = "Bạn là một trợ lý hữu ích trong lĩnh vực y tế, Nếu câu trả lời nằm ngoài phạm vi y tế và nội dung được cho, hãy trả lời không biết, không được tạo ra câu trả lời khác."
assistant = Chat(llm, retriever, prompt)

# Ví dụ sử dụng
question1 = "đau đầu là gì"
answer1 = assistant.answer(question1)
print(f"Câu hỏi: {question1}\nTrả lời: {answer1}\n")

question2 = "làm sao để trị"
answer2 = assistant.answer(question2)
print(f"Câu hỏi: {question2}\nTrả lời: {answer2}\n")

question3 = "có lưu ý nào khác không"
answer3 = assistant.answer(question3)
print(f"Câu hỏi: {question3}\nTrả lời: {answer3}\n")

# QUICK LOOK

## Augmented

In [30]:
# query = "mùa mưa hay bị bệnh gì"
# docs = retriever.invoke(query)
# pretty_print_docs(docs)

# INFO:langchain.retrievers.re_phraser:Re-phrased question: Mùa mưa có nguy cơ cao mắc các bệnh đường hô hấp.
# INFO:langchain.retrievers.multi_query:Generated queries: ['Mùa mưa có những bệnh phổ biến nào?', 'Những bệnh thường gặp vào mùa mưa là gì?', 'Tôi nên cẩn thận điều gì khi trời mưa để tránh mắc bệnh?', 'Các yếu tố môi trường trong mùa mưa có thể gây ra bệnh tật không?', 'Nguy cơ mắc bệnh tăng lên vào mùa mưa như thế nào?']
# Document 1:

# **Cách phòng ngừa**:
# -Giữ ấm cơthể, tránh dầm mưa.
# -Uống nước ấm, bổ sung thực phẩm giàu kẽm và vitamin C.
# -Vệ sinh cá nhântốt, đặc biệtlà rửa taythường xuyên.
# -Tránh tiếp xúcgầnvới ngườiđang mắc bệnh.
# ---
# ### 3. Viêm phổi (Pneumonia)
# **Mô tả**:
# Metadata: {'producer': 'macOS Version 15.4.1 (Build 24E263) Quartz PDFContext', 'creator': 'WPS Writer', 'creationdate': '2025-04-26T15:18:21+07:00', 'author': 'Hung Nguyen', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2025-04-26T15:18:21+07:00', 'sourcemodified': "D:20250426151821+07'00'", 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/source/sick in rain.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}
# ----------------------------------------------------------------------------------------------------
# Document 2:

# **Nguyên nhânliên quan đếnthờitiết**:
# -Nước mưa đọng lại ở các vật chứa (lốp xe,chậu cây) tạo môi trường lý tưởng cho muỗi
# sinh sản.
# -Độ ẩm cao và nhiệtđộ ấm trong mùa mưa giúp muỗipháttriển nhanh.
# **Cách phòng ngừa**:
# -Dọn dẹpmôi trường sống, loạibỏnước đọng.
# -Sử dụng mànchống muỗi, thuốc xịt côn trùng.
# -Mặc quần áo dài, tránh để muỗi đốt.
# -Nếucó dấuhiệusốtcao hoặc xuất huyết, đến bệnh việnngay lậptức.
# ---
# ### 5. Tiêuchảy cấp (Acute Diarrhea)
# **Mô tả**:
# Metadata: {'producer': 'macOS Version 15.4.1 (Build 24E263) Quartz PDFContext', 'creator': 'WPS Writer', 'creationdate': '2025-04-26T15:18:21+07:00', 'author': 'Hung Nguyen', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2025-04-26T15:18:21+07:00', 'sourcemodified': "D:20250426151821+07'00'", 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/source/sick in rain.pdf', 'total_pages': 3, 'page': 1, 'page_label': '2'}
# ----------------------------------------------------------------------------------------------------
# Document 3:

# -Rửa tay bằng xà phòng trước khiănvà sau khiđi vệ sinh.
# -Bù nước bằng dung dịch oresolnếu bị tiêu chảy; đếnbác sĩ nếu tình trạng kéo dài.
# ---
# ### Lưuý chung
# -**Tăng cường sức đề kháng**: Ăn uống đủ chất, bổ sung vitamin và khoáng chất,tập thể
# dục đềuđặn.
# - **Giữ vệ sinh môi trường**: Dọn dẹp nhà cửa, tránh để nước đọng, đảm bảo thông thoáng.
# -**Theo dõi sức khỏe**:Nếucó triệuchứng bấtthường như sốtcao,khó thở,hoặc tiêu
# chảy nặng,cần đikhám ngay để được điềutrịkịpthời.
# Metadata: {'producer': 'macOS Version 15.4.1 (Build 24E263) Quartz PDFContext', 'creator': 'WPS Writer', 'creationdate': '2025-04-26T15:18:21+07:00', 'author': 'Hung Nguyen', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2025-04-26T15:18:21+07:00', 'sourcemodified': "D:20250426151821+07'00'", 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/source/sick in rain.pdf', 'total_pages': 3, 'page': 2, 'page_label': '3'}
# ----------------------------------------------------------------------------------------------------
# Document 4:

# chảy nặng,cần đikhám ngay để được điềutrịkịpthời.
# -**Tham khảo y tế**: Những thông tin trên mang tínhchất thamkhảo. Nếu cần, hãy liên hệ
# bác sĩhoặc cơ sởy tế để được tư vấncụthể.
# Nếu bạncần thông tin chitiết hơnvềmột bệnh cụ thể hoặc cách điềutrị,hãy cho tôi biết!
# Metadata: {'producer': 'macOS Version 15.4.1 (Build 24E263) Quartz PDFContext', 'creator': 'WPS Writer', 'creationdate': '2025-04-26T15:18:21+07:00', 'author': 'Hung Nguyen', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2025-04-26T15:18:21+07:00', 'sourcemodified': "D:20250426151821+07'00'", 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/source/sick in rain.pdf', 'total_pages': 3, 'page': 2, 'page_label': '3'}
# ----------------------------------------------------------------------------------------------------
# Document 5:

# Phân Loại và Phân Tích
# Các từ được chia thành bốn nhóm chính: triệu chứng, bệnh lý/tình trạng, tác nhân gây bệnh, và phương pháp điều trị. Mỗi nhóm được phân tích dựa trên thông tin từ các nguồn như Wikipedia, MedlinePlus, và các tổ chức y tế quốc tế.

# Triệu Chứng
# Triệu chứng là những cảm giác chủ quan mà bệnh nhân trải nghiệm, thường là dấu hiệu của vấn đề sức khỏe tiềm ẩn. Dựa trên danh sách từ Wikipedia về triệu chứng y tế, các triệu chứng được liệt kê bao gồm:
# Metadata: {'source': '../data/source/symptom.txt'}
# ----------------------------------------------------------------------------------------------------
# Document 6:

# Ghi Chú Khảo Sát Chi Tiết
# Dưới đây là một phân tích chi tiết về 20 từ liên quan đến y tế và bệnh thông thường, dựa trên các nguồn đáng tin cậy như CDC, WHO, Mayo Clinic, và các tài liệu y khoa khác. Nội dung được tổ chức để cung cấp thông tin toàn diện, bao gồm định nghĩa, nguyên nhân, triệu chứng, điều trị, và phòng ngừa, nhằm hỗ trợ người đọc hiểu rõ hơn về các khía cạnh sức khỏe.
# Metadata: {'source': '../data/source/symptom.txt'}
# ----------------------------------------------------------------------------------------------------
# Document 7:

# cúm nhưng dễlây lan trong điềukiệnthời tiếtẩm ướt của mùa mưa.
# **Triệuchứng**:
# -Hắthơi, nghẹtmũi, chảy nước mũi.
# -Đau họng nhẹ, ho khan hoặc có đờm.
# -Mệt mỏi nhẹ,đôikhi sốtnhẹ.
# **Nguyên nhânliên quan đếnthờitiết**:
# -Độ ẩm cao và nhiệtđộ giảm độtngột làm virus dễsinh sôi.
# -Cơ thể tiếp xúc với nước mưa hoặc môi trường ẩm ướtkhiến hệ miễndịch suy yếu.
# -Virus lây lan nhanh quatiếp xúchoặc không khí trong không gian kín.
# **Cách phòng ngừa**:
# -Giữ ấm cơthể, tránh dầm mưa.
# Metadata: {'producer': 'macOS Version 15.4.1 (Build 24E263) Quartz PDFContext', 'creator': 'WPS Writer', 'creationdate': '2025-04-26T15:18:21+07:00', 'author': 'Hung Nguyen', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2025-04-26T15:18:21+07:00', 'sourcemodified': "D:20250426151821+07'00'", 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/source/sick in rain.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}
# ----------------------------------------------------------------------------------------------------
# Document 8:

# -Ở trẻem, mấtnước có thể gây nguy hiểm nếu không được bù kịpthời.
# **Nguyên nhânliên quan đếnthờitiết**:
# -Nước mưa và ngập lụt làm ônhiễm nguồn nước,thực phẩm.
# -Độ ẩm cao tạo điềukiện cho vikhuẩn và ký sinh trùng pháttriển trong thực phẩm bảo
# quảnkhông tốt.
# -Thiếu vệ sinh cá nhântrong mùa mưalàm tăng nguy cơ lây nhiễm.
# **Cách phòng ngừa**:
# -Uống nước đun sôi để nguội, tránhdùng nước không rõ nguồn gốc.
# -Rửa sạch thực phẩm,nấuchíntrước khiăn.
# -Rửa tay bằng xà phòng trước khiănvà sau khiđi vệ sinh.
# Metadata: {'producer': 'macOS Version 15.4.1 (Build 24E263) Quartz PDFContext', 'creator': 'WPS Writer', 'creationdate': '2025-04-26T15:18:21+07:00', 'author': 'Hung Nguyen', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2025-04-26T15:18:21+07:00', 'sourcemodified': "D:20250426151821+07'00'", 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/source/sick in rain.pdf', 'total_pages': 3, 'page': 2, 'page_label': '3'}
# ----------------------------------------------------------------------------------------------------
# Document 9:

# Viêm phổi là tình trạng nhiễm trùng ởphổi do vikhuẩn (như Streptococcus pneumoniae),
# virus hoặcnấm gây ra.Bệnh cóthểtrởnặng,đặc biệt ở trẻ em, ngườigià hoặc ngườicó hệ
# miễndịch yếu, vàdễ bùng phát khithờitiết chuyển mùa.
# **Triệuchứng**:
# -Sốtcao, ớn lạnh,đổ mồhôi.
# -Ho có đờm (đôi khicó máu),đau ngực khithởsâu.
# -Khó thở, thở nhanh,mệtmỏi nghiêm trọng.
# -Ở trẻnhỏ, có thểxuất hiệndấuhiệutím tái hoặc bỏbú.
# **Nguyên nhânliên quan đếnthờitiết**:
# -Độ ẩm cao trong mùa mưa tạo điềukiện cho vikhuẩn và nấm pháttriển.
# Metadata: {'producer': 'macOS Version 15.4.1 (Build 24E263) Quartz PDFContext', 'creator': 'WPS Writer', 'creationdate': '2025-04-26T15:18:21+07:00', 'author': 'Hung Nguyen', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2025-04-26T15:18:21+07:00', 'sourcemodified': "D:20250426151821+07'00'", 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/source/sick in rain.pdf', 'total_pages': 3, 'page': 1, 'page_label': '2'}
# ----------------------------------------------------------------------------------------------------
# Document 10:

# Táo bón (Constipation): Khó hoặc không thể đi ngoài, do mất nước, thiếu chất xơ. Điều trị bằng tăng nước, chất xơ, thuốc nhuận tràng nếu cần (NIDDK).

# Huyết áp (Blood pressure): Lực máu tác động lên thành động mạch, bình thường dưới 120/80 mmHg. Cao huyết áp (>130/80 mmHg) hoặc thấp (<90/60 mmHg) cần thay đổi lối sống, thuốc (AHA).
# Metadata: {'source': '../data/source/symptom.txt'}
# ----------------------------------------------------------------------------------------------------
# Document 11:

# hô hấp.Trong thời kỳ chuyển mùa, độ ẩm tăng và nhiệtđộ thay đổithất thường làm virus dễ
# lây lan quakhông khíhoặc tiếpxúc trực tiếp.
# **Triệuchứng**:
# -Sốtcao (38-40°C), ớnlạnh.
# -Đau đầu,đau cơ, mệtmỏi toàn thân.
# -Ho khan, đau họng, nghẹt mũihoặc chảy nước mũi.
# -Một số trường hợp có thể kèm buồnnôn, tiêuchảy (đặc biệtở trẻem).
# **Nguyên nhânliên quan đếnthờitiết**:
# -Độ ẩm cao trong mùa mưa giúp virus cúm tồntạilâu hơntrong không khí.
# -Thờitiếtẩm ướtkhiến cơ thể dễ bị lạnh, làm giảm sức đề kháng.
# Metadata: {'id': 9, 'relevance_score': np.float32(0.9988845), 'producer': 'macOS Version 15.4.1 (Build 24E263) Quartz PDFContext', 'creator': 'WPS Writer', 'creationdate': '2025-04-26T15:18:21+07:00', 'author': 'Hung Nguyen', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2025-04-26T15:18:21+07:00', 'sourcemodified': "D:20250426151821+07'00'", 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/source/sick in rain.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}
# ----------------------------------------------------------------------------------------------------
# Document 12:

# Dưới đây là môtảchi tiết về 5cănbệnh phổbiến mà mọi ngườidễmắc phảikhi thời tiết
# thay đổi, đặc biệt là khichuyểntừ mùa khô sangmùa mưa. Những thay đổivề độ ẩm,nhiệt
# độ và môi trường trong giai đoạnnày tạo điềukiện thuận lợi cho vikhuẩn,virus pháttriển,
# đồng thời làm suy yếu hệmiễn dịch của con người.
# ---
# ### 1. Cảm cúm (Influenza)
# **Mô tả**:
# Cảm cúm làbệnh truyền nhiễm do virus cúm (influenza virus) gâyra, ảnhhưởng đếnđường
# Metadata: {'producer': 'macOS Version 15.4.1 (Build 24E263) Quartz PDFContext', 'creator': 'WPS Writer', 'creationdate': '2025-04-26T15:18:21+07:00', 'author': 'Hung Nguyen', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2025-04-26T15:18:21+07:00', 'sourcemodified': "D:20250426151821+07'00'", 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/source/sick in rain.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}
# ----------------------------------------------------------------------------------------------------
# Document 13:

# 7. Viêm tai giữa (Otitis Media)
# Định nghĩa: Nhiễm trùng hoặc viêm tai giữa, thường gặp ở trẻ em.
# Triệu chứng: Đau tai, sốt, giảm thính lực, chảy mủ tai (nếu thủng màng nhĩ).
# Nguyên nhân: Vi khuẩn hoặc virus sau cảm lạnh, tắc ống Eustachian.
# Cách điều trị: Thuốc giảm đau, kháng sinh nếu do vi khuẩn, theo dõi (thường tự khỏi).
# 8. Phát ban nhiệt (Heat Rash)
# Định nghĩa: Tắc nghẽn ống mồ hôi gây phát ban trên da.
# Triệu chứng: Đốm đỏ nhỏ, ngứa, cảm giác châm chích ở vùng da nóng.
# Metadata: {'source': '../data/source/sick.txt'}
# ----------------------------------------------------------------------------------------------------
# Document 14:

# Nguyên nhân: Chất kích ứng (xà phòng, hóa chất) hoặc dị ứng (kim loại, thực vật như ivy độc).
# Cách điều trị: Rửa sạch vùng da, dùng kem steroid, tránh tiếp xúc với tác nhân gây bệnh.
# 6. Loét miệng (Canker Sores)
# Định nghĩa: Vết loét nhỏ, đau ở niêm mạc miệng hoặc môi.
# Triệu chứng: Đau, đỏ, vết loét trắng/vàng, khó ăn uống.
# Nguyên nhân: Không rõ, có thể do stress, chấn thương, thiếu vitamin (B12, sắt), hoặc thực phẩm chua.
# Cách điều trị: Súc miệng nước muối, dùng gel giảm đau tại chỗ, tự lành sau 1-2 tuần.
# Metadata: {'source': '../data/source/sick.txt'}
# ----------------------------------------------------------------------------------------------------
# Document 15:

# -Nhiệtđộ thay đổilàm hệ hô hấpnhạy cảm hơn, đặc biệt ởnhững người có bệnh nềnnhư
# hensuyễn hoặc COPD.
# -Mưa và ngập lụtlàm tăng nguy cơtiếp xúcvới nước bẩn,chứa vikhuẩn gây bệnh.
# **Cách phòng ngừa**:
# -Tiêm vắc-xin phòng viêm phổi (PCV, PPSV).
# -Tránh tiếp xúcvớikhóibụi, không khí ô nhiễm.
# -Giữ môi trường sống sạch sẽ, thông thoáng.
# -Tìm kiếm chăm sóc y tếngay nếucó dấuhiệusốtcao hoặc khó thở kéo dài.
# ---
# ### 4. Sốt xuất huyết (DengueFever)
# **Mô tả**:
# Metadata: {'producer': 'macOS Version 15.4.1 (Build 24E263) Quartz PDFContext', 'creator': 'WPS Writer', 'creationdate': '2025-04-26T15:18:21+07:00', 'author': 'Hung Nguyen', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2025-04-26T15:18:21+07:00', 'sourcemodified': "D:20250426151821+07'00'", 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/source/sick in rain.pdf', 'total_pages': 3, 'page': 1, 'page_label': '2'}
# ----------------------------------------------------------------------------------------------------
# Document 16:

# -Thờitiếtẩm ướtkhiến cơ thể dễ bị lạnh, làm giảm sức đề kháng.
# -Mọi ngườithường ởtrong không gian kín,tạo điều kiện cho virus lây lan quagiọtbắn.
# **Cách phòng ngừa**:
# -Tiêm vắc-xin cúm hàng năm.
# -Rửa tay thường xuyên,tránh chạm tay vào mặt.
# -Đeo khẩutrang ở nơi đông người.
# -Tăng cường sức đề kháng bằng chếđộ ăn giàuvitamin C (cam,chanh) và nghỉngơiđầy
# đủ.
# ---
# ### 2. Cảm lạnh thông thường (Common Cold)
# **Mô tả**:
# Cảm lạnh làbệnh do nhiều loại virus gây ra, phổ biếnnhất là rhinovirus. Bệnh nhẹhơn cảm
# Metadata: {'producer': 'macOS Version 15.4.1 (Build 24E263) Quartz PDFContext', 'creator': 'WPS Writer', 'creationdate': '2025-04-26T15:18:21+07:00', 'author': 'Hung Nguyen', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2025-04-26T15:18:21+07:00', 'sourcemodified': "D:20250426151821+07'00'", 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/source/sick in rain.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}

## Chat

In [ ]:
# answer log
# INFO:langchain.retrievers.re_phraser:Re-phrased question: Để truy xuất vectorstore chứa thông tin về nguyên nhân gây đau đầu, vui lòng cung cấp các truy vấn sau:

# 1. Vector chứa thông tin về căng thẳng và căng cơ làm đau đầu.
# 2. Vector chứa thông tin về migrain, một dạng headache đặc trưng bởi cơn đau mạnh mẽ thường ở một bên đầu.
# 3. Vector chứa thông tin về tension headache, nguyên nhân do căng cơ cơ thể, often caused by stress or eye strain.
# 4. Vector chứa thông tin về cluster headache, một dạng headache hiếm gặp có thể gây cơn đau dữ dội quanh hoặc sau mắt và có thể kéo dài vài giờ.
# 5. Vector chứa thông tin về sinus headache, nguyên nhân là do viêm sín, làm áp lực lên đầu.
# 6. Vector chứa thông tin về headache exertion (exercise-induced), xảy ra sau khi thực hiện運動 như vận động hoặc hoạt động căng thẳng.
# 7. Vector chứa thông tin về hormonal changes, sự biến đổi hormone estrogen và progesterone trong chu kỳ kinh nguyệt của một số phụ nữ có thể gây headache.
# 8. Vector chứa thông tin về tác dụng phụ của thuốc, một số loại thuốc như thuốc giảm đau có thể gây headache cho một số cá nhân.
# 9. Vector chứa thông tin về tai nạn đầu óc, nếu gặp phải tai nạn đầu óc thì có trường hợp rất hiếm gặp lại gây ra chứng headache mãn tính.
# 10. Vector chứa thông tin về các yếu tố khác như mất nước, huyết áp cao, thiếu ngủ, một số thức ăn hoặc allergen là một số yếu tố khác có thể là trigger cho headaches.

# Cảm ơn bạn đã cung cấp câu hỏi và hy vọng chúng tôi có thể giúp tìm kiếm vectorstore phù hợp.
# INFO:langchain.retrievers.multi_query:Generated queries: ['1. Vector chứa thông tin về căng thẳng, stress và sự cố gắng làm tăng nguy cơ đau đầu.', '2. Vector chứa thông tin về migrain, một dạng headache đặc trưng bởi cơn đau mạnh mẽ thường ở một bên đầu và kéo dài từ vài giờ đến một ngày.', '3. Vector chứa thông tin về tension headache, nguyên nhân do căng cơ cơ thể, often caused by stress hoặc eye strain và có thể gây đau đầu sau khi xem màn hình máy tính.', '4. Vector chứa thông tin về cluster headache, một dạng headache hiếm gặp có thể gây cơn đau dữ dội quanh hoặc sau mắt và kéo dài vài giờ.', '5. Vector chứa thông tin về sinus headache, nguyên nhân là do viêm sín, làm áp lực lên đầu và thường xảy ra khi chịu áp lực từ khí quản bị tắc nghẽn hoặc cảm lạnh.', '6. Vector chứa thông tin về headache exertion (exercise-induced), xảy ra sau khi thực hiện vận động như vận động hoặc hoạt động căng thẳng và có thể gây đau đầu do sự thay đổi hormone khi tập luyện.', '7. Vector chứa thông tin về hormonal changes, sự biến đổi hormone estrogen và progesterone trong chu kỳ kinh nguyệt của một số phụ nữ có thể gây headache và có liên quan đến các vấn đề kinh nguyệt.', '8. Vector chứa thông tin về tác dụng phụ của thuốc, một số loại thuốc như thuốc giảm đau có thể gây headache cho một số cá nhân và cần phải biết về khả năng gây ra cảm giác đau đầu.', '9. Vector chứa thông tin về tai nạn đầu óc, nếu gặp phải tai nạn đầu óc thì có trường hợp rất hiếm gặp lại gây ra chứng headache mãn tính và là một yếu tố nguy cơ đáng kể cho các vấn đề về đầu óc.', '10. Vector chứa thông tin về các yếu tố khác như mất nước, huyết áp cao, thiếu ngủ, một số thức ăn hoặc allergen là một số yếu tố khác có thể là trigger cho headaches và cần phải biết cách quản lý chúng.']
# Câu hỏi: đau đầu là gì??????????
# Trả lời: Đau đầu là một rối loạn thần kinh gây cảm giác đau nhói ở phần đầu. Đây thường là một sự đau dữ dội ở một bên hoặc cả hai bên của đầu và có thể làm ảnh hưởng đến khả năng sống của người bệnh. Đau nửa đầu có thể xuất hiện một cách đột ngột, làm giảm khả năng tập trung và làm ảnh hưởng đến chất lượng cuộc sống của người bệnh.

# Đau nửa đầu thường được phân loại thành "đau nhói" và "đau dữ dội". Đau nhói là cảm giác đau nhạy cảm ở phần nào đó của đầu, thường kèm theo hiện tượng co thắt cơ. Còn đau dữ dội là một cảm giác đau lớn hơn nhiều, có thể làm rung động toàn bộ đầu.

# Đau nửa đầu có thể xuất hiện một cách đột ngột và không rõ nguyên nhân, hoặc nó có thể báo hiệu sự xuất hiện của các dấu hiệu báo trước gọi là "aura". Aura thường bao gồm các triệu chứng khác như cảm giác buồn nôn, nhạy cảm với ánh sáng và âm thanh, và đôi khi có biểu hiện thần kinh (như cảm giác có vật gì đó ở đầu hoặc cảm giác bị ướt). Tuy nhiên, không phải lúc nào cũng có dấu hiệu báo trước cho một đợt đau nửa đầu dữ dội.

# Nguồn gốc của đau nửa đầu là chưa rõ, nhưng nó được cho là liên quan đến nhiều yếu tố khác nhau, bao gồm di truyền, căng thẳng, thay đổi hormone, thiếu ngủ hoặc kích thích từ thực phẩm và đồ uống như rượu và caffeine.

# INFO:langchain.retrievers.re_phraser:Re-phrased question: Tìm hiểu về cách điều trị và ngăn ngừa đau nửa đầu.
# INFO:langchain.retrievers.multi_query:Generated queries: ['1. So sánh các phương pháp điều trị đau nửa đầu phổ biến, bao gồm liệu pháp thuốc, thể thao và điều dưỡng.', '2. Phân tích các yếu tố nguy cơ và rủi ro liên quan đến đau nửa đầu và cách ngăn ngừa chúng.', '3. Tìm hiểu về sự phát triển của đau nửa đầu ở mỗi bên và các nguyên nhân có thể gây ra nó.', '4. So sánh hiệu quả của liệu pháp thuốc, trị liệu tâm lý và chăm sóc sức khỏe cơ thể đối với đau nửa đầu.', '5. Phân tích tác động của stress và căng thẳng lên khả năng mắc phải đau nửa đầu và cách giảm thiểu chúng.']
# Câu hỏi: làm sao để trị????????????
# Trả lời: Để điều trị đau nửa đầu, bạn có thể áp dụng một số cách thức sau đây:

# 1. Thuốc giảm đau: Bạn có thể sử dụng thuốc giảm đau không kê đơn (như ibuprofen) hoặc thuốc giảm đau đặc hiệu (triptans) theo chỉ dẫn của bác sĩ để kiểm soát triệu chứng.

# 2. Ngủ đủ giờ: Thay đổi lối sống và ngủ đủ giấc, từ 7-9 giờ mỗi đêm có thể giúp giảm bớt căng thẳng và mệt mỏi.

# 3. Chế độ ăn uống khoa học: Có một số thực phẩm đã được chứng minh là có thể kích hoạt các cơn đau nửa đầu, như caffeine trong cà phê hoặc trà, và tyramine trong các loại thịt đông và một số thực phẩm khác. Bạn cần tìm cách tránh xa chúng.

# 4. Chuyển đổi lối sống: Một số người bị đau nửa đầu có thể cải thiện triệu chứng của họ bằng việc thay đổi các yếu tố gây căng thẳng trong cuộc sống, như giảm áp lực làm việc hoặc học cách đối phó với căng thẳng bằng phương pháp phát triển cá nhân.

# 5. Thuốc trị liệu: Bác sĩ có thể kê toa thuốc chống stress và thuốc giảm căng thẳng cho bạn để hỗ trợ điều trị.

# 6. Massage và thực thi các bài tập thư giãn: Một số nghiên cứu chỉ ra rằng massage đầu và thực thi các bài tập thư giãn có thể giúp cải thiện triệu chứng đau nửa đầu.

# 7. Thuốc đặc trị: Bác sĩ cũng có thể kê toa thuốc đặc trị (triptans) cho bạn nếu các loại thuốc giảm đau không đủ hiệu quả.

# Lưu ý, nếu cảm thấy các triệu chứng của cơn đau nửa đầu của bạn trở nên tồi tệ hơn hoặc kéo dài quá lâu, hoặc nếu bạn bắt đầu gặp vấn đề với sự ổn định của tâm trạng và khả năng sống thường ngày, hãy liên hệ với bác sĩ để kiểm tra lại.

# INFO:langchain.retrievers.re_phraser:Re-phrased question: Tôi rất cảm kích vì sự quan tâm của bạn đến việc tìm hiểu về đau nửa đầu và cách điều trị nó. Bạn có thể hỏi về nguyên nhân chính phổ biến nhất dẫn đến đau nửa đầu, các phương pháp điều trị hiệu quả hoặc thậm chí là việc sử dụng thuốc để giảm bớt cơn đau. Miễn là vấn đề liên quan đến đau nửa đầu, tôi sẵn sàng cung cấp thông tin hữu ích và trả lời câu hỏi của bạn một cách chuyên nghiệp và lịch sự nhất.
# INFO:langchain.retrievers.multi_query:Generated queries: ['1. Tôi có thể hỏi về nguyên nhân chính phổ biến nhất dẫn đến đau nửa đầu? Bác sĩ cho biết các phương pháp điều trị hiệu quả nào cho đau nửa đầu? Còn có cách nào để sử dụng thuốc giảm cơn đau mà không gây phụ thuộc hay tác hại không mong muốn không?', '2. Rất cảm ơn bạn đã quan tâm đến cách ngăn ngừa và điều trị đau nửa đầu. Bạn biết ai là những người dễ mắc phải đau nửa đầu hơn thế nào? Có các liệu pháp thay đổi hành vi giúp giảm bớt cơn đau không?', '3. Tôi rất quan tâm đến việc tìm hiểu về nguyên nhân phổ biến của đau nửa đầu và các phương pháp điều trị hiệu quả như thế nào. Còn có cách nào để sử dụng thuốc giảm đau mà không gây phụ thuộc hay tác hại không mong muốn?', '4. Rất cảm ơn bạn đã quan tâm đến vấn đề đau nửa đầu. Ai là những người dễ mắc phải đau nửa đầu hơn? Có các liệu pháp thay đổi hành vi giúp giảm bớt cơn đau không?', '5. Bạn biết ai là những người dễ mắc phải đau nửa đầu hơn thế nào? Rất mong muốn được nghe về các phương pháp điều trị hiệu quả cho đau nửa đầu cũng như việc sử dụng thuốc giảm đau mà không gây phụ thuộc hay tác hại không mong muốn.', '6. Tôi rất quan tâm đến việc ngăn ngừa và điều trị đau nửa đầu. Ai là những người dễ mắc phải đau nửa đầu hơn? Còn có cách nào để sử dụng thuốc giảm đau mà không gây phụ thuộc hay tác hại không mong muốn?', '7. Rất cảm ơn bạn đã quan tâm đến cách ngăn ngừa và điều trị đau nửa đầu. Có các phương pháp điều trị hiệu quả cho đau nửa đầu không? Ai là những người dễ mắc phải đau nửa đầu hơn?', '8. Tôi rất quan tâm đến việc tìm hiểu về nguyên nhân phổ biến của đau nửa đầu và các phương pháp điều trị hiệu quả như thế nào. Còn có cách nào để sử dụng thuốc giảm đau mà không gây phụ thuộc hay tác hại không mong muốn? Ai là những người dễ mắc phải đau nửa đầu hơn?', '9. Rất cảm ơn bạn đã quan tâm đến vấn đề đau nửa đầu. Có các liệu pháp thay đổi hành vi giúp giảm bớt cơn đau không? Bạn biết ai là những người dễ mắc phải đau nửa đầu hơn và có các phương pháp điều trị hiệu quả cho đau nửa đầu không?', '10. Tôi rất mong muốn được nghe về nguyên nhân chính phổ biến nhất dẫn đến đau nửa đầu, các phương pháp điều trị hiệu quả và việc sử dụng thuốc giảm đau mà không gây phụ thuộc hay tác hại không mong muốn. Ai là những người dễ mắc phải đau nửa đầu hơn và có các liệu pháp thay đổi hành vi giúp giảm bớt cơn đau không?']
# Câu hỏi: có lưu ý nào khác không?????????????
# Trả lời: Có, đây là một số lưu ý khác mà bạn có thể áp dụng:

# 1. Tăng cường chế độ ăn uống lành mạnh: Đảm bảo cung cấp cho cơ thể đủ vitamin và chất dinh dưỡng cần thiết để hỗ trợ hệ thần kinh.

# 2. Giữ sự cân bằng về căng thẳng và stress: Bao quanh bản thân cách làm việc và sinh hoạt bình yên, tránh xa các nguồn gây stress cao.

# 3. Ăn không kỳ vọng: Một số nghiên cứu cho thấy việc ăn không kỳ vọng có thể giúp giảm thiểu tác động của đau nửa đầu do căng thẳng.

# 4. Sử dụng phương tiện giải trí: Hãy dành thời gian cho bản thân những hoạt động giải trí và thư giãn, ví dụ như đọc sách, nghe nhạc hoặc đi bộ trong vườn.

# 5. Kết hợp các phương pháp điều trị: Bạn có thể kết hợp sử dụng thuốc giảm đau với việc áp dụng phương pháp phát triển cá nhân để kiểm soát căng thẳng và stress.

# 6. Giữ mình khỏi kích thích: Đảm bảo tránh xa thực phẩm và đồ uống có chứa caffeine, tyramine hoặc other substances that may trigger migraines.

# 7. Chế độ sinh hoạt khoa học: Có một số bằng chứng cho thấy việc ngủ đủ giấc có thể giúp giảm triệu chứng đau nửa đầu.

# Lưu ý rằng mỗi cá nhân có những cách điều trị khác nhau hiệu quả nhất, và việc áp dụng nhiều phương pháp có thể là một chiến lược an toàn để kiểm soát triệu chứng đau nửa đầu của bạn.